In [2]:
import json
import os
import pandas as pd
import time

from dotenv import load_dotenv
from openrouteservice import client, directions, places

In [3]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1, 13): ## number of keys
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
category_id = [280] # park
buffer_size = 2000 # highest value for search radius
park_dict = {}

In [4]:
## read apartment data
property_data = json.load(open( f"{output_dir}properties_processed.json" ))
print(len(property_data['Coordinates']))

13226


In [5]:
## running from saved data
park_dict = json.load(open(f"{output_dir}closest_park.json"))['Park']

In [44]:
token = tokens[10]

## set x and y
x = len(park_dict)
y = len(park_dict)+500 ## from smallest API quota
if y > len(property_data['Coordinates'].keys()):
    y = len(property_data['Coordinates'].keys())

for index in range(x,y):
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [backwards[1], backwards[0]]
    ors = client.Client(key=token)

    ## find poi
    query = {'request': 'pois',
            'geojson': {'type':'Point', 'coordinates': coords},
            'filter_category_ids': category_id,
            'buffer': buffer_size,
            'sortby':'distance',
            'limit': 2}

    features = ors.places(**query)['features']
    
    if len(features)>0:
        destination = features[0]['geometry']['coordinates']
        all = [(str(coords[0]),str(coords[1])),(str(destination[0]),str(destination[1]))]
        ## get car route
        request = {'coordinates': all,
                'profile': 'driving-car',
                'geometry': 'true',
                'format_out':'geojson'}
        route = ors.directions(**request)
        info = route['features'][0]['properties']['summary']
    else: 
        info = {'duration':'0.0','distance':'0.0'}

    ## add information
    park_dict[str(index)] = info
    time.sleep(0.5)

IndexError: list index out of range

Stop when quota exceeds error

In [34]:
print(len(park_dict))

12486


In [35]:
property_data['Park'] = park_dict

## write json
json.dump(property_data, open(f"{output_dir}closest_park.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}closest_park.json")
data.to_csv(f"{output_dir}closest_park.csv")

In [30]:
park_dict[str(10561)] = 0 
park_dict[str(10581)] = 0
park_dict[str(10590)] = 0
park_dict[str(10591)] = 0
park_dict[str(12047)] = 0